# <img src="https://img.icons8.com/bubbles/50/000000/mind-map.png" style="height:50px;display:inline"> **ECE 046211 - Technion - Deep Learning - Final Project**
---

## **Solving Handwritten Math Exercises**
Reut Cohen & Evyatar Cohen

---
**Training YOLOv10 Model to solving handwritten math exercises**


**Imports**

In [ ]:
# imports

import zipfile
import os
import random
import shutil
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch


In [3]:
# login google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Loading Dataset For Training**

In [ ]:
# Loading Data from

# Define the path to the uploaded zip file
zip_file_path = "/content/data3.zip"  # Change this to your uploaded zip file name
# Define the extraction path
extract_path = "/content/dataset"
# Create the extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)
# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print("Dataset extracted successfully.")

Dataset extracted successfully.


In [ ]:
# Backuping Dataset

def backup_data():
    # Define the name and path of the zip file and Create a zip archive of the entire /content/ directory
    zip_filename = "/content/drive/My Drive/dataset_backup.zip"
    shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', "/content/dataset")
    print("data backed up at", time.ctime())

**Preparing data to YOLOv10 - spliting and labeling**

In [ ]:
# Preparing data to YOLO:
#Split to Test set and validation set, and make yaml txt files for the images from the contours

def create_yolo_annotations(base_path, output_images_path, output_labels_path, split_ratio):
    # make output directories for train and val sets
    os.makedirs(os.path.join(output_images_path, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_images_path, 'val'), exist_ok=True)
    os.makedirs(os.path.join(output_labels_path, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_labels_path, 'val'), exist_ok=True)

    # Define the classes
    classes = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'add', 'sub', 'mul', 'div', 'eq']

    # Process each class directory
    for class_name in classes:
        class_path = os.path.join(base_path, class_name)
        images = [img for img in os.listdir(class_path) if img.endswith(('.png', '.jpg', '.jpeg'))]

        # Shuffle images for random splitting
        random.shuffle(images)

        # Calculate split index
        split_index = int(len(images) * split_ratio)

        # Split the images into train and validation sets
        train_images = images[:split_index]
        val_images = images[split_index:]

        # Process images and create labels
        for split, img_list in zip(['train', 'val'], [train_images, val_images]):
            for image in img_list:
                image_path = os.path.join(class_path, image)
                new_image_path = os.path.join(output_images_path, split, image)
                shutil.copy(image_path, new_image_path)

                # Load the image
                img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

                # Find contours
                _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
                contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                if contours:
                    # Create a mask for all contours
                    mask = np.zeros_like(img)
                    for contour in contours:
                        cv2.drawContours(mask, [contour], -1, 255, thickness=cv2.FILLED)
                    # Find bounding boxes of all contours combined
                    x, y, w, h = cv2.boundingRect(mask)
                    # Calculate the normalized coordinates for YOLO format
                    img_h, img_w = img.shape
                    x_center = (x + w / 2) / img_w
                    y_center = (y + h / 2) / img_h
                    w = w / img_w
                    h = h / img_h

                    label_path = os.path.join(output_labels_path, split, image.replace('.jpg', '.txt').replace('.png', '.txt').replace('.jpeg', '.txt'))
                    with open(label_path, 'w') as label_file:
                        label_file.write(f"{classes.index(class_name)} {x_center} {y_center} {w} {h}\n")

# Define paths
base_path = '/content/dataset'  # Path to your original dataset sorted by folders
output_images_path = '/content/dataset/images'
output_labels_path = '/content/dataset/labels'

# Split dataset into training (90%) and validation (10%)
create_yolo_annotations(base_path, output_images_path, output_labels_path, split_ratio=0.9) # can change split ratio
backup_data()


data backed up at Sat Aug 10 18:55:38 2024


**Unzip data - If we already have the proccessed dataset**

In [1]:
# Unzipping dataset

!unzip /content/drive/My\ Drive/dataset_backup.zip -d /content/dataset

**Creating YAML file for YOLOv10**

In [ ]:
data_yaml_content = """
train: /content/dataset/images/train
val: /content/dataset/images/val

nc: 15
names: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'add' , 'sub' , 'mul' , 'div', 'eq']
"""

with open('/content/dataset/data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("data.yaml file created successfully.")

data.yaml file created successfully.


**Installing YOLOv10 Libraries**

In [ ]:
# Install necessary libraries
!pip install -q ultralytics

# Train YOLOv10
from ultralytics import YOLO
HOME = os.getcwd()
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 24.1 MB/s eta 0:00:00


**Backup trained model**

In [ ]:
# Backup Trained model

def backup_session():
    # Define the name and path of the zip file
    zip_filename = "/content/drive/My Drive/session_backup.zip"
    # Create a zip archive of the entire /content/ directory
    shutil.make_archive(zip_filename.replace('.zip', ''), 'zip', "/content/runs")
    print("Session backed up at", time.ctime())

**Training Model**

In [ ]:
# Load a model
model = YOLO(f'{HOME}/weights/yolov10m.pt')  # or path to your custom yolov10 configuration

# Train the model
model.train(data="/content/dataset/data.yaml", epochs=35, imgsz=128)


Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=/content/weights/yolov10m.pt, data=/content/dataset/data.yaml, epochs=35, time=None, patience=100, batch=16, imgsz=128, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, sh

100%|██████████| 755k/755k [00:00<00:00, 108MB/s]


Overriding model.yaml nc=80 with nc=15

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1     78720  ultralytics.nn.modules.block.SCDown          [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1    228672  ultralytic

100%|██████████| 6.25M/6.25M [00:00<00:00, 324MB/s]


AMP: checks passed ✅


...

      35/35     0.719G     0.2847     0.1595      1.647         15        128: 100%|██████████| 2289/2289 [05:17<00:00,  7.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:14<00:00,  8.86it/s]


                   all       4076       4076      0.996      0.996      0.995      0.993

35 epochs completed in 3.240 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 33.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 33.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10m summary (fused): 369 layers, 16,467,754 parameters, 0 gradients, 63.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 128/128 [00:13<00:00,  9.29it/s]


                   all       4076       4076      0.996      0.996      0.995      0.993
                     0        227        227      0.996      0.993      0.995      0.995
                     1        339        339      0.991      0.994      0.995      0.987
                     2        359        359       0.99          1      0.995      0.995
                     3        246        246      0.995          1      0.995      0.995
                     4        297        297          1      0.994      0.995      0.995
                     5        261        261      0.996      0.992      0.995      0.994
                     6        271        271          1      0.998      0.995      0.995
                     7        294        294      0.999          1      0.995      0.995
                     8        264        264      0.995          1      0.995      0.995
                     9        291        291      0.997      0.998      0.995      0.995
                   ad

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ae2a806a560>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,   

**Backup trained model**

In [ ]:
backup_session()

Session backed up at Sun Aug 11 03:25:01 2024


**Validating model**

In [ ]:
# validating model

# Validate the model
metrics = model.val()

# Print the metrics to see the performance
print(metrics)

Ultralytics YOLOv8.2.75 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
YOLOv10m summary (fused): 369 layers, 16,467,754 parameters, 0 gradients, 63.5 GFLOPs


val: Scanning /content/dataset/labels/val.cache... 4344 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4344/4344 [00:00<?, ?it/s]
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 272/272 [00:17<00:00, 15.55it/s]


                   all       4344       4344      0.996      0.993      0.995      0.992
                     0        240        240      0.995          1      0.995      0.995
                     1        364        364      0.992      0.981      0.995      0.981
                     2        383        383      0.997      0.984      0.995      0.994
                     3        259        259      0.999          1      0.995      0.995
                     4        318        318      0.997      0.995      0.995      0.995
                     5        283        283      0.996      0.997      0.995      0.994
                     6        299        299      0.997          1      0.995      0.993
                     7        325        325      0.997      0.997      0.995      0.995
                     8        278        278      0.997          1      0.995      0.994
                     9        322        322      0.994      0.997      0.995      0.995
                   ad

## <img src="https://img.icons8.com/dusk/64/000000/prize.png" style="height:50px;display:inline"> Credits
---

* Model YOLO from <a href="https://https://github.com/THU-MIG/yolov10/tree/main/docs/en">YOLO</a> - https://github.com
* Datasets from <a href="https://www.kaggle.com/">Kaggle</a> - https://www.kaggle.com/